In [ ]:
import numpy as np
import sys
sys.path.append("D:\\ASGaze")

import import_ipynb
from gaze_ray_estimator.estimator import get_cone, load_files
from mapping.mapping_principle import Line3

In [ ]:
def fit_rectangle(p1,p2,p3,flag):
        if(flag == 1):
            A = [(p2[0]+p1[0])/2,(p2[1]+p3[1])/2]
            B = [p3[0],(p2[1]+p3[1])/2]
            C = [(p2[0]+p1[0])/2,p1[1]]
            D = [p3[0],p1[1]]
        elif(flag == 2):
            A = [p1[0],(p1[1]+p2[1])/2]
            B = [(p2[0]+p3[0])/2,(p1[1]+p2[1])/2]
            C = [p1[0],p3[1]]
            D = [(p2[0]+p3[0])/2,p3[1]]
        elif(flag == 3):
            A = [p3[0],p1[1]]
            B = [(p1[0]+p2[0])/2,p1[1]]
            C = [p3[0],(p3[1]+p2[1])/2]
            D = [(p1[0]+p2[0])/2,(p3[1]+p2[1])/2]
        else:
            A = [(p2[0]+p3[0])/2,p3[1]]
            B = [p1[0],p3[1]]
            C = [(p2[0]+p3[0])/2,(p1[1]+p2[1])/2]
            D = [p1[0],(p1[1]+p2[1])/2]

        width = B[0]-A[0]
        height = C[1]-A[1]
        return np.array((A,B,C,D)),np.array((width,height))
    
def cal_offsets(PoG_CCS):
    screen_x_px = 1920
    screen_y_px = 1080
    screen_x_mm = 527
    screen_y_mm = 296

    PoG_TCS_px = [[200,150],[1650,150],[200,800],[1650,800]]
    PoG_TCS_mm = []
    
    for point in PoG_TCS_px:
        x = point[0]*screen_x_mm/screen_x_px
        y = point[1]*screen_y_mm/screen_y_px
        PoG_TCS_mm.append(np.array((x,y)))

    PoG_TCS_mm = np.array(PoG_TCS_mm)
    screen_w_h = np.array((PoG_TCS_mm[1][0]-PoG_TCS_mm[0][0], PoG_TCS_mm[2][1]-PoG_TCS_mm[0][1]))
    
    rect_1,w_h_1 = fit_rectangle(PoG_CCS[2],PoG_CCS[0],PoG_CCS[1],1)
    rect_2,w_h_2 = fit_rectangle(PoG_CCS[0],PoG_CCS[1],PoG_CCS[3],2)
    rect_3,w_h_3 = fit_rectangle(PoG_CCS[1],PoG_CCS[3],PoG_CCS[2],3)
    rect_4,w_h_4 = fit_rectangle(PoG_CCS[3],PoG_CCS[2],PoG_CCS[0],4)

    delta_WH_1 = np.sum(np.abs(w_h_1-screen_w_h))
    delta_WH_2 = np.sum(np.abs(w_h_2-screen_w_h))
    delta_WH_3 = np.sum(np.abs(w_h_3-screen_w_h))
    delta_WH_4 = np.sum(np.abs(w_h_4-screen_w_h))

    delta_WH_list = [delta_WH_1,delta_WH_2,delta_WH_3,delta_WH_4]
    min_delta,index = min(delta_WH_list),delta_WH_list.index(min(delta_WH_list))

    if(index == 0):
        offsets = np.mean((np.abs(rect_1-PoG_TCS_mm)),axis=0)
    elif(index == 1):
        offsets = np.mean((np.abs(rect_2-PoG_TCS_mm)),axis=0)
    elif(index == 2):
        offsets = np.mean((np.abs(rect_3-PoG_TCS_mm)),axis=0)
    else:
        offsets = np.mean((np.abs(rect_4-PoG_TCS_mm)),axis=0)
    
    return index+1,offsets